# WHAT DOES MY BAND LIKE TO SING ABOUT?

In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import os
from os.path import join as pjoin
import numpy as np
import pandas as pd
from time import time, sleep, strftime, localtime
from datetime import datetime

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [2]:
import re, string
import requests
from bs4 import BeautifulSoup
from itertools import groupby


In [74]:
all_songs_df = pd.read_csv("./all_song_maps.csv")
df_band_albums = pd.read_csv("./df_band_albums.csv")

In [75]:
discography_df = pd.merge(left=df_band_albums[["title", "year", "is_single"]], right=all_songs_df,left_on="title", right_on="album_name", how="inner")
discography_df.drop("album_name", axis=1, inplace=True)
display(
    discography_df.shape,
    discography_df.head()
)

(186, 5)

,title,year,is_single,song_name,song_lyrics
0,Iron Maiden,1980,False,'Prowler',"Walking through the city, looking oh so pretty\nI've just got to find my way\nSee the ladies fla..."
1,Iron Maiden,1980,False,'Sanctuary',Out of winter came a warhorse of steel\nI've never killed a woman before\nBut I know how it feel...
2,Iron Maiden,1980,False,'Remember Tomorrow',"Unchain the colours before my eyes,\nYesterday's sorrows, tomorrow's white lies.\nScan the horiz..."
3,Iron Maiden,1980,False,'Running Free',"Just sixteen, a pickup truck, out of money, out of luck.\nI've got nowhere to call my own, hit t..."
4,Iron Maiden,1980,False,'Phantom Of The Opera',I've been looking so long for you now you won't get away from my grasp\nYou've been living so lo...


# Cleaning 

In [76]:
# Remove all the escape delimiters in the lyrics
discography_df["song_lyrics"] = discography_df["song_lyrics"].apply(lambda s: re.sub("[\n\t\r]", ' ', s))

In [77]:
# What all special characters exist in the lyrics? 
unique_characters = list(set(''.join(discography_df["song_lyrics"].apply(lambda s: re.sub("[\w]", '', s)).values.ravel()[0])))
print("these are the non-alphanumeric unique characters - ", unique_characters)
discography_df["song_lyrics"] = discography_df["song_lyrics"].apply(lambda s: re.sub("[^a-zA-Z0-9]", ' ', s))

these are the non-alphanumeric unique characters -  ['?', '.', ',', "'", ' ']
